In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import mpl_lego as mplego
import pandas as pd

from hatespeech import keys, utils
from IPython.display import Markdown
from scipy.stats import bootstrap, kruskal

%matplotlib inline

In [ ]:
mplego.style.use_latex_style()

In [ ]:
exp = "08"
base_path = os.path.join(os.environ['HOME'], 'projects/annotator_bias_irt')
data_path = os.path.join(os.environ['HOME'], 'data/hatespeech/clean_qualtrics_irt_rollout.feather')
rater_quality_path = "~/data/hatespeech/rater_quality_check.csv"
results_path = os.path.join(base_path, f'scaling/experiments/exp{exp}')
out_path = os.path.join(results_path, f"exp{exp}_out.txt")

In [ ]:
with open(out_path) as f:
    lines = f.readlines()

In [ ]:
bias_lines = lines[26223:26287]

In [ ]:
races = {
    'asian': 3,
    'black': 1,
    'latinx': 2,
    'middleeastern': 4,
    'nativeamerican': 5,
    'pacificislander': 6,
    'white': 0,
    'other': 7
}
sorted_races = list(np.array(list(races.keys()))[np.argsort(list(races.values()))])

In [ ]:
n_samples = len(bias_lines)
observed = np.zeros(n_samples)
expected = np.zeros(n_samples)
bias_size = np.zeros(n_samples)
std_error = np.zeros(n_samples)
t_stats = np.zeros(n_samples)
p_vals = np.zeros(n_samples)
target = np.zeros(n_samples, dtype=str)
annotator = np.zeros(n_samples, dtype=str)
biases = np.zeros((8, 8))
std_errors = np.zeros((8, 8))
p_values = np.zeros((8, 8))

for idx, line in enumerate(bias_lines):
    processed = line.replace('|', ' ').replace('>', ' ').replace('<', ' ').split()
    observed[idx] = float(processed[0])
    expected[idx] = float(processed[1])
    bias_size[idx] = float(processed[4])
    std_error[idx] = float(processed[5])
    t_stats[idx] = float(processed[6])
    p_vals[idx] = float(processed[8])
    target[idx] = processed[13]
    annotator[idx] = processed[16]

    ii = races[line[88:105].replace(' ', '')]
    jj = races[line[112:132].replace(' ', '')]
    biases[ii, jj] = float(processed[4])
    std_errors[ii, jj] = float(processed[5])
    p_values[ii, jj] = float(processed[8])

In [ ]:
log_odds = np.exp(np.diag(biases))
upper_bound = np.exp(np.diag(biases) + np.diag(std_errors))
lower_bound = np.exp(np.diag(biases) - np.diag(std_errors))
upper_error = upper_bound - log_odds
lower_error = log_odds - lower_bound

In [ ]:
"""
Figure 4
"""
fig_height = 4.25
fig_width = 9
width = 0.2
spacing = 0.18
axis_label_size = 18
axis_tick_label_size = 16
subplot_label_size = 22
error_capsize = 4

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_height))

ax.barh(y=np.flip(np.arange(8)),
        width=log_odds,
        color='silver',
        xerr=np.concatenate(([lower_error], [upper_error]), axis=0),
        error_kw={'capsize': error_capsize},
        alpha=0.)
ax.scatter(
    log_odds,
    np.flip(np.arange(8)),
    color='k',
    s=40,
    zorder=1000)
ax.set_yticks(np.arange(8))
ax.set_yticklabels(
    np.flip(
        [r'\textbf{White}',
         r'\textbf{Black}',
         r'\textbf{Latinx}',
         r'\textbf{Asian}',
         r'\textbf{Middle Eastern}',
         r'\textbf{Native American}',
         r'\textbf{Pacific Islander}',
         r'\textbf{Other}']),
    ha='right',
    #rotation=25,
    fontsize=axis_tick_label_size)
ax.set_xlabel(r'\textbf{Change in Odds Ratio}', fontsize=axis_label_size)
ax.set_ylabel(r'\textbf{Identity Group}', fontsize=axis_label_size)
ax.set_xlim([0.80, 1.6])
ax.axvline(1, linestyle='--', color='red', lw=0.5)
ax.text(y=7.2, x=log_odds[0], s='***', va='center', ha='center', fontsize=20)
ax.text(y=6.2, x=log_odds[1], s='***', va='center', ha='center', fontsize=20)
ax.text(y=4.2, x=log_odds[3], s='**', va='center', ha='center', fontsize=20)
ax.text(y=3.2, x=log_odds[4], s='**', va='center', ha='center', fontsize=20)

ax.tick_params(labelsize=axis_tick_label_size)
ax.grid('on', axis='y')
plt.savefig('figure4.pdf', bbox_inches='tight')